In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.cluster import KMeans

# Definir la ruta base del dataset
ruta_base = 'E:\\IA'  # Cambia a la ruta de tu disco
ruta_principal = os.path.join(ruta_base, 'mammals')  # Ruta a la carpeta mammals

# Crear una lista para almacenar los arrays de imágenes, nombres de archivo y etiquetas
imagenes_data = []
nombres_archivos = []
etiquetas = []  # Para almacenar las etiquetas (nombres de las carpetas)

# Verificar que la ruta existe
if os.path.exists(ruta_principal):
    print(f'Ruta de la carpeta mammals: {ruta_principal}')
else:
    print("La ruta de mammals no existe. Verifica la ruta.")

# Iterar sobre las imágenes en el directorio y subdirectorios
for root, dirs, files in os.walk(ruta_principal):
    for archivo in files:
        if archivo.endswith('.jpg'):
            # Construir la ruta completa del archivo
            ruta_archivo = os.path.join(root, archivo)
            try:
                # Abrir la imagen
                img = Image.open(ruta_archivo)
                # Convertir la imagen a escala de grises
                img = img.convert('L')
                # Convertir la imagen a un array de numpy y aplanar
                img_array = np.array(img).flatten()
                # Añadir el array al dataset y el nombre del archivo
                imagenes_data.append(img_array)
                nombres_archivos.append(archivo)
                # Añadir la etiqueta (nombre de la carpeta)
                etiqueta = os.path.basename(root)  # Obtiene el nombre de la carpeta
                etiquetas.append(etiqueta)
            except Exception as e:
                print(f'Error al abrir {ruta_archivo}: {e}')
                continue

# Convertir la lista de imágenes a un array de numpy
data = np.array(imagenes_data)

# Aplicar KMeans para clustering
n_clusters = len(set(etiquetas))  # Cambia según el número de categorías únicas
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(data)

# Obtener etiquetas asignadas
etiquetas_kmeans = kmeans.labels_

# Crear un DataFrame con los datos, etiquetas y nombres de archivo
df = pd.DataFrame(data)  # Crear DataFrame a partir de los datos de imágenes
df['Etiqueta'] = etiquetas_kmeans  # Añadir la columna de etiquetas

# Guardar el DataFrame en un archivo CSV
df.to_csv('dataset_imagenes_con_etiquetas.csv', index=False)

print('Etiquetas guardadas en dataset_imagenes_con_etiquetas.csv')

# Separar en X (características) e y (etiquetas)
X_final = df.drop(['Etiqueta'], axis=1)  # Características (imágenes)
y_final = df['Etiqueta']  # Etiquetas

# Mostrar las dimensiones de X e y
print(f'Dimensiones de X: {X_final.shape}, Dimensiones de y: {y_final.shape}')


Ruta de la carpeta mammals: E:\IA\mammals


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42)

hola


In [ ]:
print(X_train.head())
print(X_test.head())
print(y_train.head())
print(y_test.head())

In [ ]:
probas = log_reg3.predict_proba(X_train[:1000])
# Devuelve el índice de la clase con la probabilidad más alta para cada muestra
labels_ixs = np.argmax(probas, axis=1)
# Aquí se crea un nuevo array labels que contiene las probabilidades más altas para cada muestra,
# utilizando los índices de labels_ixs
labels = np.array([proba[ix] for proba, ix in zip(probas, labels_ixs)])
# DFinalmente, se seleccionan las 10 muestras con las probabilidades más bajas (es decir, las más inciertas)
# al indexar labels con los primeros 10 índices de sorted_ixsevuelve los índices que ordenarían el
# array labels en orden ascendente.
sorted_ixs = np.argsort(labels)
labels[sorted_ixs[:10]]

In [ ]:
X_lowest = X_train.values[500:1000][sorted_ixs[:k]]

# Asegúrate de que k sea un número positivo
k = min(k, len(X_lowest))  # Asegúrate de que k no exceda la cantidad de imágenes a mostrar

# Crea la figura para las imágenes
plt.figure(figsize=(15, 8))  # Aumenta el tamaño de la figura

# Determina el número de filas y columnas
n_cols = 2  # Cambia a 5 columnas para mostrar mejor
n_rows = (k // n_cols) + (k % n_cols > 0)  # Calcula el número de filas

# Itera sobre las imágenes seleccionadas
for index, img in enumerate(X_lowest[:k]):  # Asegúrate de limitar a k imágenes
    plt.subplot(n_rows, n_cols, index + 1)  # Usa n_rows y n_cols
    plt.imshow(img.reshape(64, 64), cmap="binary", interpolation="bilinear", aspect='auto')  # Cambia a 64x64
    plt.axis('off')  # Elimina los ejes

# Ajustar el espaciado entre subgráficas
plt.tight_layout()  # Esto ayuda a evitar superposiciones
plt.show()

In [ ]:
# Obtiene las primeras 1000 etiquetas del conjunto de entrenamiento
# Es un array que contiene los índices de las probabilidades clasificadas
y_lowest = y_train.values[:1000][sorted_ixs[:k]]
# seleccionas las etiquetas que corresponden a esas imágenes de baja probabilidad
y_lowest

In [ ]:
# es un array que contiene las etiquetas propagadas para las imágenes de entrenamiento (etiquetas asignadas
# automaticamente)
y_train2 = y_train_propagated[:1000].copy()
# sorted son los índices de las imágenes con las probabilidades más bajas de clasificación
# estás reemplazando las etiquetas de las imágenes de baja probabilidad (que están en y_train2 en esos índices)
# con las nuevas etiquetas que has obtenido y almacenado en y_lowest
y_train2[sorted_ixs[:k]] = y_lowest

In [ ]:
log_reg5 = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=4000, random_state=42)
%time log_reg5.fit(X_train[:1000], y_train2)
log_reg5.score(X_test, y_test)